## CS5803 NLP
### Assignment 2
#### Tanmay Garg, Tanmay Goyal, Tanay Yadav
#### Roll no: CS20BTECH11063, AI20BTECH11021, AI20BTECH11026

##### Link to dataset: https://www.kaggle.com/datasets/moxxis/harry-potter-lstm.

In [26]:
# importing necessary packages
import re
from collections import defaultdict
from math import log, exp  
import numpy as np
import random
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tanmaygoyal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tanmaygoyal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##### **Q1. Preprocess and tokenize the dataset using NLTK**

In [27]:
# reading the original data
with open('Harry_Potter_all_char_separated.txt', 'r', encoding='utf-8') as file:
    harry_potter_data = file.read()

def preprocess_text(text):
    '''
    Function to preprocess the text by removing punctuations and converting to lower case
    '''
    # [^\w\s] -> ^ means except , \w refers to any alphanumeric character and \s refers to whitespace    
    text = re.sub(r'[^\w\s]', '', text).lower()
    return text

def tokenize(text):
    '''
    Function to tokenize the text
    '''
    tokens = nltk.tokenize.word_tokenize(text)
    # stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokens = [token for token in tokens if token not in stop_words]
    return tokens

# preprocessing the text
harry_potter_text = preprocess_text(harry_potter_data)
# using the first 10000 words
harry_potter_tokens = tokenize(harry_potter_text)[:10000]

##### **Q2. Fit two bigram language models on the text: MLE and kneserNey Discounting**

In [28]:
# fit two bigram language models on the text: MLE and Kneser-Ney discounting using the nltk library

def MLE_bigram(n , n_gram , vocab):
    '''
    Function to fit a bigram language model using MLE
    '''
    model = nltk.lm.MLE(n)
    model.fit([n_gram], vocabulary_text = vocab)
    return model

def KN_bigram(n , n_gram , vocab):
    '''
    Function to fit a bigram language model using Kneser-Ney discounting
    '''
    model = nltk.lm.KneserNeyInterpolated(n)
    model.fit([n_gram], vocabulary_text = vocab)
    return model

harry_potter_bigrams = nltk.ngrams(harry_potter_tokens, 2)

# converting the bigrams to a list
bigrams = []
for bigram in harry_potter_bigrams:
    bigrams.append(bigram)

bigram_mle = MLE_bigram(2 , bigrams , harry_potter_tokens)
bigram_kn = KN_bigram(2 , bigrams , harry_potter_tokens)

##### **Q3. Use the beginning words 1. "Harry Potter" and 2. "Dumbledore" to generate text using both the language models. Keep maximum text length as 20**

In [29]:
def generate_prediction(model , num_words = None , text_seed = None , random_seed = None):
    '''
    Function to generate the predictions given text_seed and num_words.
    It joins them together in a sentence and returns the sentence
    '''

    # preprocessing the text_seed
    text_seed = preprocess_text(text_seed)
    text_seed = tokenize(text_seed)
    
    # generating the prediction
    pred = model.generate(num_words = num_words , text_seed = text_seed , random_seed = random_seed)
    
    sentence = text_seed + [pred[i] for i in range(num_words)]
    predicted_sentence = sentence[0]
    for word in sentence[1:]:
        predicted_sentence += ' ' + word
    return predicted_sentence


print("==== MLE Model ====")
print(generate_prediction(bigram_mle , num_words = 20 , text_seed = "Harry Potter" , random_seed = 123))
print(generate_prediction(bigram_mle , num_words = 20 , text_seed = "Dumbledore" , random_seed = 123)) 

print("==== KneserNey Model ====")
print(generate_prediction(bigram_kn , num_words = 20 , text_seed = "Harry Potter" , random_seed = 123))
print(generate_prediction(bigram_kn , num_words = 20 , text_seed = "Dumbledore" , random_seed = 123)) 

==== MLE Model ====
harry potter are are less like us all right of them angrily it he does tend to be nice if hed just
dumbledore and bacon as dudley was a man had swapped at his favorite program had expected mrs dursleys bought dudley he
==== KneserNey Model ====
harry potter are are less like us all right place you cant ive finished dialing his hair and piers and had four
dumbledore and bacon as dudley was a mad old things gray tuesday our heads down old clothes of arms and james


Beam search is a tree-based search strategy similar to BFS. In BF, we expand every child node, however, in Beam Search, we only expand the top k most probable children. The generated text is the text with the highest probabiltity

##### **Q4. To implement beam search, implement a function to find the top k most probable words**

In [32]:
def k_top_probable(model , k , word):
    '''
    returns the top-k most probable words based on the model given
    ''' 
    
    # we store the non-zero probabilities
    non_zero_prob = {}

    for w in model.vocab:
        if model.score(w , [word]) > 0:
            non_zero_prob[w] = model.score(w , [word])

    # sorting the non_zero_prob based on values
    sorted(non_zero_prob.items() , key = lambda item:item[1])
    
    if len(non_zero_prob) > k:
        return list(non_zero_prob.keys())[:k]
    
    else:
        top = list(non_zero_prob.keys())

        # we sample words randomly which had zero probability
        random = []
        for i in range(k - len(non_zero_prob)):
            w = random.choice(model.vocab)
            while w in top or w not in random:
                w = random.choice(model.vocab)
            random.append(w)

        return top + random

##### **Q5. Implement Beam Search using the MLE language model previously trained.**

##### **Q6. Repeat Q3 using Beam Search with k=2 and depth = 10. Find the 5 generated texts with highest probability for each of the three sentences.**